In [ ]:

!apt-get install poppler-utils

In [ ]:

!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain==0.0.173 --progress-bar off
!pip install -qqq chromadb==0.3.23 --progress-bar off
!pip install -qqq pypdf==3.8.1 --progress-bar off
!pip install -qqq pygpt4all==1.1.0 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off

In [3]:
!gdown 1DpFisoGXsQbpQJvijuvxkLW_pg-FUUMF

Downloading...
From: https://drive.google.com/uc?id=1DpFisoGXsQbpQJvijuvxkLW_pg-FUUMF
To: /content/ms-financial-statement.pdf
100% 29.1k/29.1k [00:00<00:00, 74.8MB/s]


In [4]:
!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin

--2023-09-21 05:11:25--  https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin
Resolving gpt4all.io (gpt4all.io)... 104.26.1.159, 172.67.71.169, 104.26.0.159, ...
Connecting to gpt4all.io (gpt4all.io)|104.26.1.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3785248281 (3.5G)
Saving to: ‘ggml-gpt4all-j-v1.3-groovy.bin’

ggml-gpt4all-j-v1.3 100%[===================>]   3.52G  23.1MB/s    in 97s     

2023-09-21 05:13:03 (37.1 MB/s) - ‘ggml-gpt4all-j-v1.3-groovy.bin’ saved [3785248281/3785248281]



In [5]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path

In [6]:
from langchain.document_loaders import TextLoader

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
loader = TextLoader("/content/drive/MyDrive/limited_liability-drive/matched_txt_20210908/01-27-15 MSA.txt")

In [9]:
documents = loader.load_and_split()

In [10]:
len(documents)

8

In [11]:

print(documents[0].page_content)

yy
ay?

MASTER SERVICES AGREEMENT

This Master Services Agreement (this “Agreement”), is made as of January 1, 2015 (the “Effective Date”), by and
between Novitex Enterprise Solutions, Inc., a Delaware corporation, with a principal place of business at 300 First Stamford

Place, Second Floor West, Stamford, Connecticut 06902 (‘“Novitex’’), and Blackhawk Network, Inc., an Arizona corporation
with a place of business at 6220 Stoneridge Mall Road, Pleasanton, CA 94588 (“Customer”).

SECTION 1. Scope. During the Term (defined below), the parties may enter into one or more statements(s) of work, each
of which will become effective when signed by both parties (each, a “SOM”)., Each SOW will set forth the services to be
provided (the “Services”), the fees for the Services and any other applicable terms. Each SOW will be governed by this

Agreement; if there is a conflict between a SOW and this Agreement, the SOW will control.

SECTION 2. Standards of Performance; Warranty. The Services will b

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(documents)

In [13]:
texts[0:2]

[Document(page_content='yy\nay?\n\nMASTER SERVICES AGREEMENT\n\nThis Master Services Agreement (this “Agreement”), is made as of January 1, 2015 (the “Effective Date”), by and\nbetween Novitex Enterprise Solutions, Inc., a Delaware corporation, with a principal place of business at 300 First Stamford\n\nPlace, Second Floor West, Stamford, Connecticut 06902 (‘“Novitex’’), and Blackhawk Network, Inc., an Arizona corporation\nwith a place of business at 6220 Stoneridge Mall Road, Pleasanton, CA 94588 (“Customer”).\n\nSECTION 1. Scope. During the Term (defined below), the parties may enter into one or more statements(s) of work, each\nof which will become effective when signed by both parties (each, a “SOM”)., Each SOW will set forth the services to be\nprovided (the “Services”), the fees for the Services and any other applicable terms. Each SOW will be governed by this\n\nAgreement; if there is a conflict between a SOW and this Agreement, the SOW will control.', metadata={'source': '/cont

In [14]:

len(texts)

40

In [15]:
print(texts[0].page_content)

yy
ay?

MASTER SERVICES AGREEMENT

This Master Services Agreement (this “Agreement”), is made as of January 1, 2015 (the “Effective Date”), by and
between Novitex Enterprise Solutions, Inc., a Delaware corporation, with a principal place of business at 300 First Stamford

Place, Second Floor West, Stamford, Connecticut 06902 (‘“Novitex’’), and Blackhawk Network, Inc., an Arizona corporation
with a place of business at 6220 Stoneridge Mall Road, Pleasanton, CA 94588 (“Customer”).

SECTION 1. Scope. During the Term (defined below), the parties may enter into one or more statements(s) of work, each
of which will become effective when signed by both parties (each, a “SOM”)., Each SOW will set forth the services to be
provided (the “Services”), the fees for the Services and any other applicable terms. Each SOW will be governed by this

Agreement; if there is a conflict between a SOW and this Agreement, the SOW will control.


In [16]:
# creating embedding

In [17]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [18]:

db = Chroma.from_documents(texts, embeddings, persist_directory="db")

In [19]:
#Create Chain
model_n_ctx = 1000
model_path = "./ggml-gpt4all-j-v1.3-groovy.bin"
llm = GPT4All(model=model_path, n_ctx=1000, backend="gptj", verbose=False)

In [20]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    verbose=False,
)


In [ ]:
#Ask Questions
%%time
res = qa(
    "what is standards of Performance; Warranty.? Extract as it is from the text."
)